# House Prices Prediction

In [2]:
# Import libraries
import numpy as numpy
import pandas as pd
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
# Folder path 
folder_path = 'D:\FPTUni\SP24\ADY201m\Lab05\ADY201m_Lab05_SE183256'
submit_path = 'D:\FPTUni\SP24\ADY201m\Lab05\ADY201m_Lab05_SE183256'
# Import the dataset
train_data = pd.read_csv(os.path.join(folder_path, 'train.csv'))
test_data = pd.read_csv(os.path.join(folder_path, 'test.csv'))
submit_file = os.path.join(folder_path, 'submission.csv')

In [4]:
# Define some constants
threshold = 0.5



In [5]:
# Because MSSubClass is int64 but it is a categorical variable, so we need to convert it to string
train_data['MSSubClass'] = train_data['MSSubClass'].astype(str)
# Drop Id column
train_data.drop('Id', axis=1, inplace=True)

In [6]:
# Get the categorical columns
categorical_cols = train_data.select_dtypes(include=['object']).columns

In [7]:
# Label encoding for categorical columns, keep dataframes type
label_encoder = LabelEncoder()
for col in categorical_cols:
    train_data[col] = label_encoder.fit_transform(train_data[col])

In [8]:
# Compute the correlation matrix with SalePrice
correlation_matrix = train_data.corrwith(train_data['SalePrice']).sort_values(ascending=False)
# Print the correlation matrix
# ABS correlation matrix and choose the features have correlation >= threshold
selected_features = correlation_matrix[abs(correlation_matrix) >= threshold].index.tolist()

In [9]:
# Get the selected features to new dataframe
prelast_train_data = train_data[selected_features]

In [10]:
# Khởi tạo scaler
scaler = StandardScaler()
# Build scaler dành riêng cho tập output
scaler_output = StandardScaler()
scaler_output.fit(prelast_train_data['SalePrice'].values.reshape(-1, 1))
# Fit scaler trên tập train
scaler.fit(prelast_train_data)
# Transform tập train
last_train_data = scaler.transform(prelast_train_data)

In [27]:
# Convert the numpy array to torch tensor
torch.set_printoptions(precision=10)
last_train_data = torch.tensor(last_train_data, dtype=torch.float32)

# Prepare the training set, last_train_data is the scaled training set and it is a tensor now
X_pretrain = last_train_data[:, 1:]
y_pretrain = last_train_data[:, 0]

# Split the train data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X_pretrain, y_pretrain, test_size=0.1, random_state=42)

C:\Users\ASUS\AppData\Local\Temp\ipykernel_2440\1217171245.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  last_train_data = torch.tensor(last_train_data, dtype=torch.float32)


In [55]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))  # Lớp ẩn 1 với hàm kích hoạt ReLU
        x = self.relu(self.fc2(x))  # Lớp ẩn 2 với hàm kích hoạt ReLU
        x = self.sigmoid(self.fc3(x))  # Lớp đầu ra với hàm kích hoạt Sigmoid
        return x

# Dữ liệu mẫu
input_size = 14  # Số lượng đặc trưng đầu vào
hidden_size1 = 16  # Số lượng units trong lớp ẩn 1
hidden_size2 = 8  # Số lượng units trong lớp ẩn 2
output_size = 1  # Số lượng output (trong bài toán hồi quy)

# Khởi tạo mô hình MLP
model = MLP(input_size, hidden_size1, hidden_size2, output_size)

In [56]:
# Di chuyển mô hình và dữ liệu vào GPU nếu có sẵn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
X_train = X_train.to(device)
y_train = y_train.to(device)

# Xác định hàm mất mát và thuật toán tối ưu hóa
criterion = nn.MSELoss()  # Hàm mất mát Mean Squared Error (MSE) cho bài toán hồi quy
optimizer = optim.SGD(model.parameters(), lr=0.01)  # Thuật toán tối ưu hóa SGD với learning rate 0.01

# Huấn luyện mô hình
num_epochs = 5000
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(X_train)

    # Tính toán loss
    loss = criterion(outputs, y_train.unsqueeze(1))

    # Backward pass và cập nhật trọng số
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # In ra thông tin về quá trình huấn luyện
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

model.to("cpu")
X_train = X_train.to("cpu")
y_train = y_train.to("cpu")

Epoch [10/5000], Loss: 1.1645
Epoch [20/5000], Loss: 1.1525
Epoch [30/5000], Loss: 1.1406
Epoch [40/5000], Loss: 1.1288
Epoch [50/5000], Loss: 1.1170
Epoch [60/5000], Loss: 1.1052
Epoch [70/5000], Loss: 1.0934
Epoch [80/5000], Loss: 1.0815
Epoch [90/5000], Loss: 1.0696
Epoch [100/5000], Loss: 1.0577
Epoch [110/5000], Loss: 1.0458
Epoch [120/5000], Loss: 1.0338
Epoch [130/5000], Loss: 1.0220
Epoch [140/5000], Loss: 1.0103
Epoch [150/5000], Loss: 0.9988
Epoch [160/5000], Loss: 0.9873
Epoch [170/5000], Loss: 0.9761
Epoch [180/5000], Loss: 0.9651
Epoch [190/5000], Loss: 0.9544
Epoch [200/5000], Loss: 0.9441
Epoch [210/5000], Loss: 0.9342
Epoch [220/5000], Loss: 0.9246
Epoch [230/5000], Loss: 0.9155
Epoch [240/5000], Loss: 0.9069
Epoch [250/5000], Loss: 0.8987
Epoch [260/5000], Loss: 0.8910
Epoch [270/5000], Loss: 0.8837
Epoch [280/5000], Loss: 0.8769
Epoch [290/5000], Loss: 0.8706
Epoch [300/5000], Loss: 0.8646
Epoch [310/5000], Loss: 0.8589
Epoch [320/5000], Loss: 0.8537
Epoch [330/5000],

In [39]:
# Đánh giá mô hình trên tập test bằng mean squared error (MSE)
with torch.no_grad():
    outputs = model(X_test)
    y_test_tensor = y_test.unsqueeze(1)[:outputs.size(0)]  # Đảm bảo kích thước của y_test phù hợp với outputs
    mse = criterion(outputs, y_test_tensor)
    # In ra kết quả MSE dạng phần trăm
    print(f'Mean Squared Error (MSE) trên tập test: {mse.item()*100:.2f}%')



Mean Squared Error (MSE) trên tập test: 90.08%


In [14]:
# Setup for test_data

test_data = pd.read_csv(os.path.join(folder_path, 'test.csv'))
# Convert the test_data to same format as train_data
test_data['MSSubClass'] = test_data['MSSubClass'].astype(str)

# Save ID column for submission
test_data_ID = test_data['Id']
test_data.drop('Id', axis=1, inplace=True)

# Label encoding for categorical columns, keep dataframes type but test_data doesn't have SalePrice column
# So we will build new categorical_cols for test_data
selected_features2 = selected_features.copy()
selected_features2.remove('SalePrice')
for col in selected_features2:
    test_data[col] = label_encoder.fit_transform(test_data[col])

# Get the selected features to new dataframe
prelast_test_data = test_data[selected_features2]

# Fit scaler trên tập test
scaler.fit(prelast_test_data)

# Transform tập test
last_test_data = scaler.transform(prelast_test_data)

# Convert the numpy array to torch tensor
last_test_data = torch.tensor(last_test_data, dtype=torch.float32)

In [15]:
# Prepare the test set, last_test_data is the scaled test set and it is a tensor now
X_test = last_test_data

# Forward pass
y_pred = model(X_test)

# Chuyển kết quả dự đoán về dạng numpy array
y_pred = y_pred.detach().numpy()

# Chuyển kết quả dự đoán về dạng chuẩn
y_pred = scaler_output.inverse_transform(y_pred)

# Làm tròn kết quả dự đoán
y_pred = y_pred.round()

# Tạo dataframe chứa kết quả dự đoán
submission = pd.DataFrame({'Id': test_data_ID, 'SalePrice': y_pred.flatten()})

# Xuất dataframe ra file CSV
submission.to_csv(submit_file, index=False)

# In ra thông báo hoàn thành
print(f"Đã hoàn thành xuất file {submit_file}")

Đã hoàn thành xuất file D:\FPTUni\SP24\ADY201m\Lab05\ADY201m_Lab05_SE183256\submission.csv


In [ ]:
# Kiểm tra kết quả dự đoán bằng tập X_test